# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [17]:
# !python -m pip install -r requirements.txt

In [18]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.config.add_text_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_text_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills

In [19]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [20]:
skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")

Define your ASK. What do you want the Kernel to do?

In [5]:
ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other."
original_plan = await planner.create_plan_async(ask, kernel)

In [6]:
print(original_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan becomes the `input` to the next function. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [7]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=1000, temperature=0.8)

Let's update our ask using this new skill.

In [11]:
ask = """Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other"""

new_plan = await planner.create_plan_async(ask, kernel)

In [12]:
print(new_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "ShakespeareSkill.shakespeare"},
            {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}}
        ]
    }


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [10]:
results = await planner.execute_plan_async(new_plan, kernel)

In [11]:
print(results)


Hi there,

Are you looking for a romantic activity to do with your special someone? Here are some ideas that might be fun: 

1. Cook a romantic banquet for two.
2. Enjoy a picnic in the park. 
3. Attend a night of painting. 
4. Visit a nearby museum. 
5. Make a pleasurable journey. 
6. Attend a jocose show. 
7. Take a cooking class together. 
8. Go for a walk on the shore. 
9. Watch a movie at the drive-in. 
10. Play a game night.

Thanks,
